# Omnibus Tests and Model Comparisons
In the previous section, we saw how to use dummy variables with multiple categorical levels and then specify a subsequent regression model that fits group means and mean differeces. Although we called this a One-way ANOVA model, it probably does not look much like an ANOVA to you yet. In this section, we will complete the picture by discussing the nature of the ANOVA omnibus tests and showing how these are effectively *model comparisons*. The familiar ANOVA table is simply a way of displaying the results of several model comparisons. We will show how to generate this within `R` and also show how the values within this table relate to comparing different regression models.

## ANOVA as a Model Comparison Procedure
In order to understand the ANOVA within the framework of linear models, we need to demonstrate how the ANOVA results are simply the outcomes from comparing different linear models in a specific way. To begin, we need to consider the logic of an *omnibus test*. 


### Omnibus Tests
An omnibus test is a test that contains *multiple* comparisons between means. In comparison to a procedure such as a $t$-test, which only compares *two* means, an omnibus test can compare *multiple* means. In our example of a One-way ANOVA where $k = 3$, the omnibus null hypothesis is

$$
\mathcal{H}_{0} : \mu_{1} = \mu_{2} = \mu_{3}.
$$

In other words, the null is that *all* the means are identical. Or, to put it another way, that all the mean differences are 0. An omnibus test of this hypothesis is able to simultaneously compare *all possible mean differences*. Within the NHST framework, a significant omnibus effect suggests that *at least one* of all possible mean differences is significant. Traditionally, we would then drill-down to see which of the differences is driving the omnibus effect.

At this point, you may well ask: what is the point of an omnibus test? If we just end up drilling-down to work out which differences are significant, why not just start there? Why bother with omnibus tests at all? The traditional argument is one of *error control*, which requires us to take a quick detour into the world of *multiple comparisons*.

...

### Omnibus Tests From Model Comparisons
So, how do we generate omnibus tests from model comparisons? To see how this works, consider that the omnibus null hypothesis above actually *implies* a specific model. If it were true that all the group means are identical, then the grouping variable is effectively meaningless. It has simply chopped the data up randomly into 3 groups. Assuming that the population means of the groups are all the *same* implies that all the data are drawn from the *same population distibution*. Thus, rather than assuming

$$
y_{ij} \sim \mathcal{N}\left(\mu_{j},\sigma^{2}\right)
$$

we are assuming

$$
y_{i} \sim \mathcal{N}\left(\mu,\sigma^{2}\right).
$$

Here we have *two different models*. One that just assumes a single mean for all the data and one that assumes *different* means for the different groups. In the context of a regression model, this is a comparison between a model that only contains an intercept, and a model containing an intercept *and* dummy variables for the categories.

To see this more clearly, we will call these models the *null model* and the *full model*. In `R`, sticking with out `mtcars` example, we could specify these in the following way

In [1]:
data(mtcars)
mtcars$origin <- c('Japan','Japan','USA','USA','USA','USA','USA','Europe','Europe',
                   'Europe','Europe','Europe','Europe','Europe','USA','USA','USA',
                   'Europe','Japan','Japan','Japan','USA','USA','USA','USA',
                   'Europe','Europe','Europe','USA','Europe','Europe','Europe')
mtcars$origin <- as.factor(mtcars$origin)

null.mod <- lm(mpg ~ 1,      data=mtcars)
full.mod <- lm(mpg ~ origin, data=mtcars)

Thus, the full model contains our categorical variable of interest and the null model does not. If our question revolves around whether this predictor is actually necessary, a natural way to do so would be to compare how well each model fits the data. To do so, we can calculate the sum-of-squared residuals as an (un-normalised) measure of the variance left over by each model[^SS-foot]. The large the value, the more variance remains and the *worse* the model fits the data.

In [2]:
null.SS <- sum(resid(null.mod)^2)
full.SS <- sum(resid(full.mod)^2)
print(c(null.SS,full.SS))

[1] 1126.0472  732.1721


So, we can see already that the null model has a much larger residual variance and thus is worse fit to the data. The difference between these values therefore tells us how much the error reduces after including `origin` in the model. This difference is known as the *between-groups sum-of-squares*, or $SS_{B}$ for short

In [3]:
SS.B <- null.SS - full.SS
print(SS.B)

[1] 393.8751


On its own, this value does not mean much. However, what we want to do is see whether the amount we have managed to explain by including `origin` in the model is large or small relative to how much variability we still have left. If adding `origin` to the model has done very little, then we will still have a lot of variability left over. However, if the means of the groups truly do differ, then adding `origin` should have soaked up quite a lot of variability and we will not have as much left over. As such, we want to compare $SS_{B}$ to the sum-of-squared residuals from the full model. In other words, the (un-normalised) measure of variance left over once `origin` has been added to the model. This is known as the *within-group sum-of-squares*, or $SS_{W}$ for short

In [4]:
SS.W <- full.SS
print(SS.W)

[1] 732.1721


### The $F$-ratio
Given what we have discussed above, we could just eye-ball the magnitude of $SS_{B}$ relative to $SS_{W}$. However, a more principled approach is to combine these values into some sort of *test statistic*, akin to the $t$-statistic we have discussed previously. In doing so, we can combine the two sources of information into a single standardised value that can be interpreted across different datasets. Much in the same way that a $t$-statistic is formed by dividing an *effect* by *error*, we could similarly form a statistic by calculating $\frac{SS_{B}}{SS_{W}}$. This would tell us how large the improvement in model fit is, relative to the amount of error left over. 

As an example, if $\frac{SS_{B}}{SS_{W}} = 1$ then the model improvement has the same magnitude as the error. In other words, the improvement is no bigger than what you would expect just from noise. In this situation we would typically *discount* any improvement as evidence of a genuine effect[^noise-foot]. However, if $\frac{SS_{B}}{SS_{W}} = 2$ then the amount we have reduced the error is *twice* the amount left over. In other words, the improvement is much larger than we would expect just from noise. In this situation, we would typically conclude that a genuine effect has been found because the possibility that we are only seeing noise becomes less and less plausible. 

The statistic we use for this purpose is known as $F$ (after *Fisher*). Given the description above, it may seem like we could calculate $F = \frac{SS_{B}}{SS_{W}}$. However, there is some additional complexity. As calculated above, $SS_{B}$ and $SS_{W}$ are not directly comparable, for reasons we will now discuss. 

#### Mean-squares and the $F$-ratio
Beginning with $SS_{B}$, we can interpret this value as the improvement in model fit (the reduction in error) after adding a certain number of parameters to the model. Unfortunately, this value will always scale with the number of parameters we add This is because the more parameters we add, the better the fit can become[^params-foot]. Because of this, we do not know whether $SS_{B}$ is large because of a genuine improvement, or simply because we have added many more parameters. To make this value more meaningful, we can therefore divide it by the number of parameters we have added to give an *average improvement per-parameter*. The number of additional parameters is known as the *numerator degrees of freedom* (or $df_{1}$) and the scaled version of $SS_{B}$ is known as the *between-group mean square*, or $MS_{B}$ for short. As such 

$$
MS_{B} = \frac{SS_{B}}{df_{1}}.
$$

In the case of the `mtcars` example, we need to add $k - 1 = 2$ parameters to the model to capture the different group means. As such, we divide $SS_{B}$ by $df_{1} = 2$ to produce the $MS_{B}$

In [5]:
df.1 <- 2
MS.B <- SS.B / df.1
print(MS.B)

[1] 196.9376


So, we have an improvement in model fit in terms of reducing the error variance by an average of 196.94 for each parameter we have added.


Moving on to $SS_{W}$, we have a similar problem because this value always scales with sample size. ...

We can therefore think of the two mean-square values as directly comparable, because they both represent *average variances* after taking into account the elements of the model that scale their magnitude.

So, in actual fact, $MS_{W}$ is simply estimating $\sigma^{2}$, Bessel's correction and all. 

In [ ]:
df.2 <- full.mod$df.residual
MS.W <- SS.W / df.2
print(MS.W)

[1] 25.24731


Compared with

In [7]:
sigma <- summary(full.mod)$sigma
print(sigma^2)

[1] 25.24731


#### The Definition of $F$

... So, the $F$-ratio is compaing two measures of variance. Hence the name, Analysis of *Variance*. 



$$
F = \frac{\text{Average reduction in variance per-parameter}}{\text{Average variance left over per-observation}}.
$$

Or, to put this a difference way, how much have we managed to explain for each parameter we have added compared to how much we still have left to explain? The larger the $F$-ratio becomes, the greater the variance reduction relative to the error and the smaller the associated $p$-value becomes.

A summary of all the calculations involved in producing an $F$-ratio that compares the null and full model in order to test the omnibus hypothesis about group means is shown below

In [17]:
# Null model and full model
null.mod <- lm(mpg ~ 1,      data=mtcars)
full.mod <- lm(mpg ~ origin, data=mtcars)

# Residual sums-of-squares
null.SS  <- sum(resid(null.mod)^2)
full.SS  <- sum(resid(full.mod)^2)

# Between-group sums-of-squares (model improvement)
SS.B <- null.SS - full.SS

# Within-group sums-of-squares (error variance)
SS.W <- full.SS

# Mean-squares
df.1 <- 2
df.2 <- full.mod$df.residual
MS.B <- SS.B / df.1
MS.W <- SS.W / df.2

# F-ratio
F <- MS.B / MS.W
p <- pf(q=F, df1=df.1, df2=df.2, lower.tail=FALSE)

# Results
print(c(F,p))


[1] 7.800337890 0.001946794


In order to compare the $SS_{B}$ and $SS_{W}$, we typically calculate a *test statistic* called $F$ (named after *Fisher*), which has a known distribution under the null. This test statistic takes the form of a *ratio* between two variance terms. The logic is to compare the amount of variance we have *explained* to the amount of variance that remains *unexplained*. This effectively gives a *signal-to-noise* ratio. The larger the value of $F$, the larger the "signal" is (the variance we have explained) relative to the "noise" (the variance we have not explained). In the context of the one-way ANOVA, this "signal" is based on assuming that the group means actually differ. If this assumption results in a much larger improvement in terms of explained variance relative to error variance, it suggests that the full model fits the data better than the null model. The alternative is that the improvement in explained variance is of a similar magnitude to the error variance. In this situation, we cannot tell whether any "improvement" is just noise, because it looks effectively the same as the error. As such, if the null were true, we would expect $F \approx 1$. This is because there are no differences between the group means and so both $SS_{B}$ and $SS_{W}$ are capturing noise. The larger the value of $F$ becomes, the less we believe the null because the magnitude of improvement is much greater than we would expect if it were just noise. In this case, $SS_{W}$ is still capturing noise, but $SS_{B}$ is capturing something more systematic about the data. Any time we get $F > 1$ this is suggestive that the null may not be correct[^NHST-foot]. The larger $F$ becomes, the greater this suggestion becomes and the smaller the associated $p$-value. 

## ANOVA Tables in `R`

### The `anova()` Function

In [8]:
print(anova(null.mod, full.mod))

Analysis of Variance Table

Model 1: mpg ~ 1
Model 2: mpg ~ origin
  Res.Df     RSS Df Sum of Sq      F   Pr(>F)   
1     31 1126.05                                
2     29  732.17  2    393.88 7.8003 0.001947 **
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Within this table, we should see all the values that we calculated manually above, alongside the $p$-value for the $F$-ratio.

### The `Anova()` function

In [9]:
library(car)
print(Anova(full.mod))

Loading required package: carData



Anova Table (Type II tests)

Response: mpg
          Sum Sq Df F value   Pr(>F)   
origin    393.88  2  7.8003 0.001947 **
Residuals 732.17 29                    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


... Notice that this table says `Type II tests` at the top. This is something we will explore in more detail in the associated synchronous session next week.

... So, the core understanding here is that the ANOVA is simply a *model comparison procedure*. The ANOVA effects usually reported within an ANOVA table are really comparisons between a model that contains that effect and a model that does not contain that effect. The *variance* within the name ANOVA is a comparison between the error variances of these models. In other words, the ANOVA is asking the question "how much does the error reduce when we include this effect in the model?" The table is just a helpful way of organising all these comparisons. So we can see that the ANOVA model is in fact just a form of multiple regression, and the tests associated with the ANOVA are summaries of model comparisons. This is the true way to conceptualise an ANOVA.

## The Regression ANOVA $F$-test
... In fact, this test is already provided at the bottom of the regression table...

Note that this will not always align with the ANOVA tests we want. For simple models that only contain a single factor and *nothing else*, the regression ANOVA test will be equivalent to an omnibus test on that single factor. For models that contain *multiple* factors and/or continuous predictors, this will not be equivalent. As such, it can be good practise to produce the ANOVA table anyway, even if in some cases it is redundant.

## Follow-up Tests

## The General ANOVA Procedure
At this point, we have examined a lot of theory and output releated to ANOVA models. This can make these models look very complicated, whereas the actual procedure inside `R` is actually very simple. As shown below, we generally just follow 5 steps:

1. Convert categorical variables from strings into factors.
2. Fit and summarise the model using `lm()`.
3. Generate omnibus tests using the `Anova()` function from the `car` package.
4. Generate follow-up tests using the `emmeans` package.
5. Extract and plot the fitted values using the `effects` package.

Indeed, these same 5 steps can be followed for *every linear model you ever use*. The only differences are that steps 1, 3 and 4 are not needed when there are no categorical predictors in the model. Obviously, you also need to check assumptions, consider transformations and do all the other things we discussed last week. In addition, we have options at different stages in terms of the exact output we need. For instance, we can choose to generate NHST results from `emmeans`, or generate confidence intervals instead. However, the core of the modelling procedure exists within these 5 steps.

To see this in action, consider the following summary of the complete `mtcars` analysis

In [10]:
library(car)
library(effects)
library(emmeans)

data(mtcars)
mtcars$origin <- c('Japan','Japan','USA','USA','USA','USA','USA','Europe','Europe',
                   'Europe','Europe','Europe','Europe','Europe','USA','USA','USA',
                   'Europe','Japan','Japan','Japan','USA','USA','USA','USA',
                   'Europe','Europe','Europe','USA','Europe','Europe','Europe')

# Step 1 - convert categorical predictors
mtcars$origin <- as.factor(mtcars$origin)

# Step 2 - fit the model
origin.mod <- lm(mpg ~ origin, data=mtcars)
print(summary(origin.mod))

# Step 3 - generate the ANOVA table
anova.tbl <- Anova(origin.mod)
print(anova.tbl)

# Step 4 - generate follow-up tests
follow.up <- emmeans(origin.mod, specs=pairwise ~ origin, adjust='holm')
print(follow.up$contrasts)

# Step 5 - extract and plot effects
effs <- allEffects(origin.mod)
plot(effs)

lattice theme set by effectsTheme()
See ?effectsTheme for details.



ERROR: Error in library(emmeans): there is no package called ‘emmeans’


This leads to a reasonable amount of output, but which contains everything we need to reach conclusions. Indeed, the summarising of the results of `lm()` is not strictly necessary. However, it is good practice to always look at this to make sure the model is correct, otherwise we can miss important misspecifications that will cause issues later on. It is also notable that the actual reference level of the dummy variables does not matter here, nor does even understanding the coding used for the dummy variables. These are *implementation details* that matter for understanding the theory, but should not influence the final results.

[^params-foot]: One way to think about this is the extreme example of having one parameter for every observation. In this case, we could make the model fit *perfectly* as each parameter can make sure that the exact value of the raw data is predicted. If we do this, there will be 0 error. Thus, we can always make the model better and better by adding more and more parameters. However, this is an extreme example of *over-fitting*, where the model will be capturing *noise* and not any universal truth about the data in question. 

[^SS-foot]: A lot of the ANOVA theory involves messing around with sums-of-squares, mean squares and degrees of freedom and it may not always be clear *why* each is being used at each stage. For instance, why use sums-of-squares rather than just calculating the error variance directly? The simple answer is that all of this complexity comes down to requiring a test statistic with a null distribution that is tractable. In other words, this was the only way that these results were possible in a world before computers. These days, if you were using a method such as permutation testing, none of this is necessary as the tractability of the null does not matter.

[^NHST-foot]: With all the caveats around the logic of NHST still in place.

[^noise-foot]: You may wonder why we cannot just conclude that the improvement is very small, rather than assuming it is noise simply because it is of the same magnitude. The point here is the ability to *distinguish* genuine effects from noise. If they are similar magnitude then we cannot know which is which. As such, even if the effect is genuine but small, we cannot be certain that it is not just noise and thus cannot reject the null.